# About
Implement clustering algorithms
1. K-Means
1. Heirarchical clustering
1. DBSCAN

## The Team
| Name| Student ID|
|------------|---------------|
|Cynthia Cai | 5625483 |
|Pratyush Kumar | 5359252|


# Imports

// add the imports to the cell below

In [1]:
import numpy as np 
import pandas as pd
import scipy.spatial
from scipy.spatial import ConvexHull, distance_matrix
from sklearn.metrics.pairwise import euclidean_distances as eucDist
import glob
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

# Reading the dataset


From the readme for the xyz files, we know that:

Ground truth labels:
|File range|Label|
|--|--|
|    000 - 099: |building|
|    100 - 199: |car|
|    200 - 299: |fence|
|    300 - 399: |pole|
|    400 - 499: |tree|


workflow:

iterate through the files, and collect them in a dataframe

Use [this link](https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat) for concatenating the dataframes

In [2]:
xyzPath = './scene_objects/data/*.xyz'

dataPathsList = glob.glob(xyzPath)

In [3]:
allPointsDF= pd.DataFrame(columns=['x','y','z', 'fileNo', 'groundLabel'])
# featureDF = pd.DataFrame(columns=['Label' , 'convHull', median] )

def df_maker(df1, df2):
    return pd.concat([df1, df2], sort=False, ignore_index=True)

labelToGive = None
for path in dataPathsList:
    indx = int(path.split('\\')[-1][0:3])
    # if else to determine label
    if indx>=0 and indx<100:
        labelToGive = 'building' 
    elif indx>=100 and indx<200:
        labelToGive = 'car' 
    elif indx>=200 and indx<300:
        labelToGive = 'fence' 
    elif indx>=300 and indx<400:
        labelToGive = 'pole' 
    elif indx>=400 and indx<500:
        labelToGive = 'tree' 

    # print(indx, labelToGive)        

    # using pandas to read dataset and make a dataFrame
    tempDF = pd.read_csv(path, delimiter=' ', header=None, dtype=np.float64, names=['x','y','z'])
    tempDF.loc[:,'fileNo'] = indx
    tempDF.loc[:,'groundLabel'] = labelToGive

    # merge with megaDFofPoints
    allPointsDF = df_maker(allPointsDF, tempDF)

# allPointsDF.head()

In [4]:
# save to pickle file
# allPointsDF.to_pickle('./scene_objects/compressedData.pkl')

## Making feature points
Identified feature points: `//add more`
* median height(z)
* convex hull

In [4]:
def label_determiner(indx):
    labelToGive=None
    if indx>=0 and indx<100:
        labelToGive = 'building' 
    elif indx>=100 and indx<200:
        labelToGive = 'car' 
    elif indx>=200 and indx<300:
        labelToGive = 'fence' 
    elif indx>=300 and indx<400:
        labelToGive = 'pole' 
    elif indx>=400 and indx<500:
        labelToGive = 'tree' 
    return labelToGive


featureDF = allPointsDF.groupby('fileNo').var()
featureDF.rename(columns={'x':'varX','y':'varY','z':'varZ'}, inplace=True)
featureDF.loc[:,'median_Z'] = allPointsDF.groupby('fileNo').z.median()
# featureDF.loc[:,'mean_Z'] = allPointsDF.groupby('fileNo').z.mean()

# range of x,y,z
featureDF.loc[:,'range_X'] = allPointsDF.groupby('fileNo').x.max() - allPointsDF.groupby('fileNo').x.min()
featureDF.loc[:,'range_Y'] = allPointsDF.groupby('fileNo').y.max() - allPointsDF.groupby('fileNo').y.min()
featureDF.loc[:,'range_Z'] = allPointsDF.groupby('fileNo').z.max() - allPointsDF.groupby('fileNo').z.min()

featureDF.loc[:,'Volume'] = allPointsDF.set_index('fileNo').loc[:,'x':'z'].groupby('fileNo').apply(ConvexHull).apply(lambda x: x.volume)

# points density
featureDF.loc[:,'footprintDensity'] =  allPointsDF.groupby('fileNo').count().x / (featureDF.range_X * featureDF.range_Y)
featureDF.loc[:,'volumeDensity'] =  allPointsDF.groupby('fileNo').count().x / featureDF.Volume

featureDF.loc[:,'label'] = featureDF.reset_index().fileNo.apply(label_determiner)

# standardize DF
standardFeatureDF = (featureDF.iloc[:,:-1] - featureDF.iloc[:,:-1].mean() ) / featureDF.iloc[:,:-1].std()

# join labels to the feature DF
standardFeatureDF = standardFeatureDF.join(other=featureDF.label ,on='fileNo')

featureDF.to_pickle('./scene_objects/featureData.pkl')
standardFeatureDF.to_pickle('./scene_objects/standardFeatureData.pkl')

featureDF

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
0,9.024868,1.760537,0.501052,17.92,10.480000,4.649994,5.02,104.189009,37.121449,17.362676,building
1,7.306054,2.628307,0.646622,7.53,10.540009,6.139984,3.60,137.366395,32.078876,15.112867,building
2,19.973520,18.707730,2.108935,13.35,17.039997,16.059998,7.49,1247.880682,32.711848,7.173763,building
3,27.224888,16.674539,2.437923,14.43,21.160004,16.750000,7.07,1326.712538,29.001490,7.747722,building
4,30.802399,22.456995,0.597981,7.80,23.579994,22.090012,5.21,1100.901866,23.277809,11.013697,building
...,...,...,...,...,...,...,...,...,...,...,...
495,2.468837,1.670906,2.625734,7.14,6.969986,5.820001,7.19,138.917876,67.520725,19.716685,tree
496,6.586047,4.938702,7.407274,9.84,10.640015,10.010002,12.44,645.231457,62.822416,10.369922,tree
497,1.379584,2.840469,2.370287,7.38,5.279999,6.920013,6.84,84.578072,31.994384,13.821549,tree


### Plotting to see resemblamces and clusters, if any
needed: seaborn

In [5]:
# load df's
featureDF = pd.read_pickle('./scene_objects/featureData.pkl')
standardFeatureDF = pd.read_pickle('./scene_objects/standardFeatureData.pkl')

In [ ]:
sns.pairplot(data=featureDF, hue="label")

normalize the feature df </br>
[from stackoverflow we see](https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame), that we can just use pandas for a standard scaling, or else, a [standard scaler from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) can also be applied </br>

from [answer here](https://stats.stackexchange.com/questions/417339/data-standardization-vs-normalization-for-clustering-analysis), we see that standard scaler is used for k means , so we are going with that

In [ ]:
sns.pairplot(data=standardFeatureDF, hue="label")

# Clustering Algorithms
note: already loaded the featureDF and standardised in the cell above

## K-Means clustering

In [101]:
def kmeans(featureDF, k):
    """
    Using DBScan method to cluster feature points.
    Input parameter:
        featureDF: a DataFrame that stores feature points with label
        k: the number of clusters
    Output:
        cluster: a list of dataframes
        C = [C0,C1,C2,C3,C4]
        Cx is a DataFrame with the same column of featureDF
    """

    #pre-step: setting parameters and format conversion
    MaxInteration, epsilon = 50, 0.0000000000000000000000000000000000000000000000000000000000001

    column_name = standardFeatureDF.columns.values.tolist()
    col_num = len(column_name) # the number of features

    pts_label_array = standardFeatureDF.to_numpy() # has 500 elements
    pts_array = standardFeatureDF.loc[:,'varX':'volumeDensity'].to_numpy()

    cluster = [pd.DataFrame(columns = column_name)] * k # a list of dataframes(cluster)
    # Step1: initialize k centroids (array)
    centroid_idx = np.random.randint(0,len(pts_array),k).tolist()
    centroid_array = pts_array[centroid_idx]

    # step2: assign each point to a cluster
    interation = 0
    centroid_change = []
    while interation <= MaxInteration:
        interation += 1

        cls_0, cls_1, cls_2, cls_3, cls_4 = [], [], [], [], []

        tree = scipy.spatial.cKDTree(centroid_array)
        for j in range(0,len(pts_array)):
            pt = pts_array[j]
            d, i = tree.query(pt) # i is the index of centroid
            
            if i == 0:
                cls_0.append(j)
            elif i == 1:
                cls_1.append(j)
            elif i == 2:
                cls_2.append(j)
            elif i == 3:
                cls_3.append(j)
            elif i == 4:
                cls_4.append(j)
            
        cluster = []
        cluster.append(standardFeatureDF.iloc[cls_0])
        cluster.append(standardFeatureDF.iloc[cls_1])
        cluster.append(standardFeatureDF.iloc[cls_2])
        cluster.append(standardFeatureDF.iloc[cls_3])
        cluster.append(standardFeatureDF.iloc[cls_4])

        # step3: update centroids 更新均值向量
        old_centroid_array = centroid_array
        #print(interation," centroid",old_centroid_array)
        for i in range(0,k):
            centroid_array[i] = cluster[i].mean(axis = 0).to_numpy() # the input is a list of arrays
            #print(centroid_array[i])
            dist = scipy.spatial.distance.euclidean(old_centroid_array[i], centroid_array[i])
            if dist <= epsilon:
                break
    
    print("Iteration:", interation)
    return(cluster)


In [195]:
kmeans_cls = kmeans(standardFeatureDF, 5)

C:\Users\cynthia\AppData\Local\Temp/ipykernel_40880/998090184.py:62: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  centroid_array[i] = cluster[i].mean(axis = 0).to_numpy() # the input is a list of arrays


Iteration: 51


In [196]:
kmeans_cls[0]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
6,-0.113495,-0.118877,-0.549384,-0.560106,-0.064492,-0.013431,-0.928677,-0.245457,0.729352,1.418477,building
65,-0.149758,-0.118020,-0.554631,-0.599026,-0.316758,-0.011346,-0.963699,-0.266881,0.763156,1.602550,building
79,-0.145686,0.000542,-0.543329,-0.689030,-0.261918,0.375636,-0.889278,-0.242471,0.734804,1.445283,building
101,-0.178768,-0.198209,-0.568329,0.517503,-0.661473,-0.388938,-0.961510,-0.299167,0.075355,1.457934,car
102,-0.162454,-0.226749,-0.555714,1.091579,-0.424874,-0.692472,-0.922111,-0.296234,0.408744,1.032873,car
...,...,...,...,...,...,...,...,...,...,...,...
271,-0.033046,-0.232354,-0.560550,-1.090397,0.257504,-0.801993,-0.954943,-0.294125,-0.311923,0.513440,fence
274,-0.061073,-0.156246,-0.564256,-1.306891,0.260636,-0.139642,-0.963699,-0.298858,-1.454523,0.249715,fence
283,-0.043975,-0.216464,-0.570728,-1.542846,0.215197,-0.587120,-0.930866,-0.297895,-1.237790,0.445461,fence


In [197]:
kmeans_cls[1]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
1,-0.118543,-0.186223,-0.463418,-0.567404,-0.016701,-0.301320,-0.504040,-0.201540,0.446179,-0.151984,building
2,-0.009724,0.104511,-0.196479,0.848326,0.492535,0.733408,0.347423,0.634529,0.485685,-0.634648,building
3,0.052568,0.067749,-0.136423,1.111039,0.815313,0.805380,0.255491,0.693879,0.254109,-0.599754,building
4,0.083300,0.172302,-0.472297,-0.501725,1.004905,1.362382,-0.151635,0.523874,-0.103126,-0.401196,building
5,-0.142520,-0.003433,-0.509320,0.597776,-0.167121,0.541483,-0.482151,-0.128204,0.167088,-0.194485,building
...,...,...,...,...,...,...,...,...,...,...,...
495,-0.160097,-0.203533,-0.102139,-0.662272,-0.296391,-0.334697,0.281758,-0.200372,2.658229,0.127909,tree
496,-0.124728,-0.144448,0.770711,-0.005490,-0.008866,0.102351,1.430905,0.180814,2.364991,-0.440335,tree
497,-0.169454,-0.182386,-0.148770,-0.603891,-0.428791,-0.219957,0.205148,-0.241283,0.440906,-0.230490,tree


In [198]:
kmeans_cls[2]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
30,0.025439,0.128938,-0.331429,1.597544,0.559128,0.971228,0.347423,0.626987,0.333725,-0.582889,building
54,0.135278,0.177598,0.330677,1.488080,0.914810,0.864836,1.109144,1.770825,0.639731,-0.769425,building
76,0.119911,0.260013,0.219632,0.833731,0.949282,0.988962,1.082877,2.071540,0.554138,-0.795135,building
92,0.113950,0.080148,-0.157371,1.765388,0.772224,0.816853,0.323346,0.742426,0.368063,-0.602664,building
300,-0.165002,-0.216161,0.714719,0.529665,-0.100528,-0.468210,1.076311,-0.237604,-1.314996,-0.957978,pole
...,...,...,...,...,...,...,...,...,...,...,...
470,-0.090862,-0.096477,0.472572,0.901842,0.285706,0.464297,1.700133,0.463743,0.970312,-0.602950,tree
471,-0.151427,-0.179214,1.023586,0.485880,-0.090345,-0.041595,1.903697,0.076194,1.141019,-0.640875,tree
481,-0.131860,-0.094470,2.096495,2.329734,-0.040987,0.400671,2.389621,0.440464,1.670400,-0.652874,tree


In [199]:
kmeans_cls[3]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
0,-0.103778,-0.201913,-0.489991,1.959990,-0.021402,-0.456737,-0.193223,-0.226519,0.760904,-0.015205,building
33,-0.140919,-0.147776,-0.182274,2.169188,-0.218046,-0.153202,0.218281,-0.107749,0.409041,-0.630501,building
57,-0.035693,0.205777,-0.178350,1.536731,0.432994,1.164197,0.119783,0.490302,0.222855,-0.531609,building
67,0.089730,-0.137785,-0.184038,1.923503,0.655491,-0.092703,0.270813,0.376703,1.122434,-0.622312,building
77,-0.111529,0.087477,-0.418682,1.945395,-0.020619,0.618670,0.270813,0.257587,0.750770,-0.598870,building
91,-0.100064,0.124862,-0.358384,2.130267,-0.005733,0.712546,0.251114,0.387663,0.872623,-0.635224,building
97,-0.059681,0.030607,-0.098544,2.239731,0.310777,0.475769,0.257681,0.231299,0.193707,-0.592124,building
100,-0.165070,-0.228047,-0.570862,0.967520,-0.446027,-0.693513,-1.018420,-0.300700,-0.778581,0.541225,car
156,-0.159740,-0.227633,-0.564020,1.069686,-0.334778,-0.683081,-0.941810,-0.295912,-0.394738,0.442004,car


In [200]:
kmeans_cls[4]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
15,0.529247,0.835365,-0.172228,0.167219,1.703735,2.075841,0.802704,4.027059,0.193375,-0.792343,building
22,0.487179,0.069621,0.006603,-0.056573,1.876875,0.660394,0.487510,1.587554,0.559182,-0.633801,building
34,0.300241,0.196379,-0.360664,1.086714,1.392709,1.228867,0.535665,1.074351,-0.155294,-0.628638,building
52,0.132543,0.259343,0.167663,0.369119,1.076198,1.074495,0.754550,1.847550,0.132408,-0.798475,building
64,0.708257,0.785378,-0.128269,0.191544,2.105640,2.071667,0.769871,4.312063,0.057280,-0.792212,building
73,0.268994,0.381835,0.223355,0.247492,1.399759,2.034118,0.942791,1.906312,-0.288258,-0.727490,building
201,0.256100,0.589329,-0.564593,-0.608756,1.139658,1.221568,-0.873956,-0.013547,-1.437863,-0.914816,fence
204,1.432276,1.954636,-0.558610,-0.934715,3.977279,2.463865,-0.698848,1.290510,-1.489698,-1.009591,fence
210,0.487152,-0.229255,-0.540533,-0.810656,1.905863,-0.501589,-0.652882,-0.212951,-1.208993,-0.661364,fence


## Heirarchical clustering

This [ref was nice](https://www.section.io/engineering-education/hierarchical-clustering-in-python/) for heirarchical clustering understanding
Some other sources:
* [Statquest](https://www.youtube.com/watch?v=7xHsRkOdVwo&ab_channel=StatQuestwithJoshStarmer)
* Penn state [pseudo code](https://online.stat.psu.edu/stat508/lesson/12/12.7)
* pseudo code from [researchgate](https://www.researchgate.net/figure/The-hierarchical-clustering-algorithm-in-pseudocode_fig1_202144697)
* towards data science article to do [step by step](https://towardsdatascience.com/breaking-down-the-agglomerative-clustering-process-1c367f74c7c2) {this is a good one to follow}
* another one [for theory](https://towardsdatascience.com/machine-learning-algorithms-part-12-hierarchical-agglomerative-clustering-example-in-python-1e18e0075019)
* similar [theory as above](https://www.geeksforgeeks.org/ml-hierarchical-clustering-agglomerative-and-divisive-clustering/)
* real good [step by step explaination](https://medium.com/@darkprogrammerpb/agglomerative-hierarchial-clustering-from-scratch-ec50e14c3826), also the [github code](https://github.com/Darkprogrammerpb/DeepLearningProjects/blob/master/Project40/agglomerative_hierarchial_clustering/Hierarchial%20Agglomerative%20clustering.ipynb)

### To Think in heirarchical clustering:
* Which type of heirarchical clustering are we doing: lets begin with agglomerative clustering
* Within the selected type what distance metrics are we using


In [ ]:

tempDF = standardFeatureDF.iloc[:,:-1].copy()

def heirarch_clust(dataDF):
    distances = eucDist(standardFeatureDF.drop('label', axis=1))
    
    pass


# calculate distances
# maybe change the distance computation
distMatDF = pd.DataFrame( distance_matrix(tempDF.values, tempDF.values), index = tempDF.index, columns = tempDF.index)
# distMatDF = pd.DataFrame( np.tril(distMatDF),  index = tempDF.index, columns = tempDF.index)
distMatDF = distMatDF.where(distMatDF!=0, np.nan)
distMatDF


devise new distance matrix and then repeat the sequence:
### TODO: 
* linkage between the clusters
* updation of the distance matrix

clusters to be made:
`vals.idxmin()` and `idVals.iloc[vals.idxmin()]`

In [4]:
tempDF = standardFeatureDF.iloc[:,:-1].copy()

distMatDF = pd.DataFrame( distance_matrix(tempDF.values, tempDF.values), index = tempDF.index, columns = tempDF.index)
# distMatDF = pd.DataFrame( np.tril(distMatDF),  index = tempDF.index, columns = tempDF.index)
# replace 0 distances with np.nan
distMatDF = distMatDF.where(distMatDF!=0, np.nan)
    
clusterKeeper = {}
clustDict={}
clusterKeeperList = []
clustCheck = {}
# clustCHECK WILL have two nodes each
iterationCounter=0
play=[]
m=len(distMatDF)
progression = [ [i] for i in range(m) ] 

while m>1: 

    # cluster size
    # print(f"Total sample = {m}")
    # compute distances

    # get indices with min dist
    vals = distMatDF.min(skipna=True)
    idVals = distMatDF.idxmin(skipna=True)

    # print(vals.min(), vals.idxmin()) # GIVES US THE MINIMUM VALUE and the index at which this was found in the vals series
    # print(idVals.iloc[vals.idxmin()])
    
    ind_to_pop = [idVals.loc[vals.idxmin()] , vals.idxmin()]
    # print(f"index {ind_to_pop}")
    play.append(ind_to_pop)
    # update distmatrix at some point
    # add updated new row, col to dist mat  
    # this updated row is basically the minimum of the two eliminated rows
    singleLink_minRow = distMatDF.loc[ind_to_pop].drop(ind_to_pop, axis=1).max()
    singleLink_minRow.rename(f"cluster {iterationCounter}", inplace=True)

    # pop row and col from dist mat
    distMatDF = distMatDF.drop(ind_to_pop, axis=0).drop(ind_to_pop, axis=1)
    # print("row,col ",len(distMatDF),len(distMatDF.columns))

    # min distance from other points

    distMatDF = distMatDF.append(singleLink_minRow)
    distMatDF.loc[:,singleLink_minRow.name] = singleLink_minRow
    # update value of m
    m = len(distMatDF)
    # m-=1
    clusterKeeper[f"iteration {iterationCounter}"] = {'indices_popped':ind_to_pop , "df":distMatDF.copy()}
    clusterKeeperList.append( (iterationCounter, ind_to_pop) )
    clustDict[f"cluster {iterationCounter}"] = ind_to_pop
    
    indPop1, indPop2 = ind_to_pop

    clustCheck[f"cluster {iterationCounter}"] = {'node1':indPop1 , "node2":indPop2, 'fullnodes':ind_to_pop}
    print("before" , clustCheck[f'cluster {iterationCounter}'])
    
    # Case: if first index is a cluster
    if (indPop1 in clustCheck.keys()) and (indPop2 in clustCheck.keys()): #both are clusters
        clustCheck[f"cluster {iterationCounter}"] = {'node1':clustCheck[indPop1]['fullnodes'].copy() , "node2":clustCheck[indPop2]['fullnodes'].copy() }
        tempFull = clustCheck[f"cluster {iterationCounter}"]["node1"].copy()
        # try:
        tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"].copy()) #if it is a list
        # except:
        #     tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"]) # if it isnt a list and thus can't be copied
        clustCheck[f"cluster {iterationCounter}"]["fullnodes"] = tempFull  


    # Case: if first index is a cluster
    elif indPop1 in clustCheck.keys(): #means first position is cluster
        clustCheck[f"cluster {iterationCounter}"] = {'node1':clustCheck[indPop1]['fullnodes'].copy() , "node2":indPop2 }
        tempFull = clustCheck[f"cluster {iterationCounter}"]["node1"].copy()
        try:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"].copy()) #if it is a list
        except:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"]) # if it isnt a list and thus can't be copied
        clustCheck[f"cluster {iterationCounter}"]["fullnodes"] = tempFull

    # Case: if second index is a cluster
    elif indPop2 in clustCheck.keys(): #means first position is cluster
        clustCheck[f"cluster {iterationCounter}"] = {'node1':indPop1 , "node2":clustCheck[indPop2]['fullnodes'].copy()}
        tempFull = clustCheck[f"cluster {iterationCounter}"]["node1"].copy()
        try:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"].copy())
        except:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"])

        clustCheck[f"cluster {iterationCounter}"]["fullnodes"] =  tempFull

    print("after" , clustCheck[f'cluster {iterationCounter}'])

    iterationCounter+=1
distMatDF

before {'node1': 151, 'node2': 142, 'fullnodes': [151, 142]}
after {'node1': 151, 'node2': 142, 'fullnodes': [151, 142]}
before {'node1': 198, 'node2': 123, 'fullnodes': [198, 123]}
after {'node1': 198, 'node2': 123, 'fullnodes': [198, 123]}
before {'node1': 160, 'node2': 136, 'fullnodes': [160, 136]}
after {'node1': 160, 'node2': 136, 'fullnodes': [160, 136]}
before {'node1': 389, 'node2': 370, 'fullnodes': [389, 370]}
after {'node1': 389, 'node2': 370, 'fullnodes': [389, 370]}
before {'node1': 148, 'node2': 135, 'fullnodes': [148, 135]}
after {'node1': 148, 'node2': 135, 'fullnodes': [148, 135]}
before {'node1': 175, 'node2': 167, 'fullnodes': [175, 167]}
after {'node1': 175, 'node2': 167, 'fullnodes': [175, 167]}
before {'node1': 96, 'node2': 83, 'fullnodes': [96, 83]}
after {'node1': 96, 'node2': 83, 'fullnodes': [96, 83]}
before {'node1': 194, 'node2': 103, 'fullnodes': [194, 103]}
after {'node1': 194, 'node2': 103, 'fullnodes': [194, 103]}
before {'node1': 185, 'node2': 104, 'ful

fileNo,cluster 498
fileNo,
cluster 498,NaN


## DBSCAN

In [84]:
import random

In [139]:
def dbscan(featureDF, radius, MinPts):
    """
    Using DBScan method to cluster feature points.
    Input parameter:
        featureDF: a DataFrame that stores feature points with label
        radius: the radius of search circle 
        MinPts: the minimum number of neighbor points required for a core point
    Output:
        cluster: a list of dataframes
        C = [C0,C1,C2,C3,C4]
        Cx is a DataFrame with the same column of featureDF
    """
    
    # pre-step: data structure and kdtree
    column_name = featureDF.columns.values.tolist()
    pts_label_list = featureDF.to_numpy().tolist()

    pts_DF = featureDF.loc[:,'varX':'volumeDensity']
    pts_array = pts_DF.to_numpy() # an array of arrays (all pts)

    tree = scipy.spatial.cKDTree(pts_array)

    # step1: generate core points
    wait_idx = [*range(0,len(pts_DF))] # a list index (wait pts)
    core_idx = [] # a list of index (core pts)
    print(len(core_idx), " core points.")
    for i in wait_idx:
        pt = pts_array[i]
        neighbor = tree.query_ball_point(pt, radius) # a list of indexs of neighbors
        if len(neighbor) >= MinPts:
            core_idx.append(i) 

    # step2: pick a random core point and start the loop (using queue)
    # initialize cluster number and waiting list
    k = 0
    cluster = []

    while len(core_idx) > 0:
        clusterDF = pd.DataFrame(columns=column_name)
        
        start_idx = random.sample(core_idx,1)[0]
        queue = [start_idx]
        core_idx.remove(start_idx)
        wait_idx.remove(start_idx)
        clusterDF.append(pts_label_list[start_idx])

        # step3: process each pt in queue
        # first of all: core or non-core?
        # for a non-core point, assign it to a cluster (done)
        # for a core neighbor point, assign it to a cluster (done) and add its neighbors to queue
        while len(queue) > 0:
            pt = pts_array[queue.pop(0)] 
            neighbor_idx = tree.query_ball_point(pt, radius) # a list of indexs (neighbor pts)       
            
            if len(neighbor_idx) >= MinPts:
                for index in neighbor_idx:
                    if index in wait_idx: # 将 在邻域中 & 未处理 的点 
                        queue.append(index) # 加入队列
                        wait_idx.remove(index) # 移除出 等待集
                        clusterDF.loc[len(clusterDF.index)] = pts_label_list[index] # 加入 聚类簇
                        if index in core_idx:
                            core_idx.remove(index) # 移除出 核心集
        
        # step4: store the new cluster
        k = k + 1
        cluster.append(clusterDF)

        if k > 10:
            break
            print("DBScan clusters out of ten!")

    return(cluster)

In [194]:
dbscan_cluster = dbscan(standardFeatureDF,0.6,5)

dbscan_cluster

0  core points.


[         varX      varY      varZ  median_Z   range_X   range_Y   range_Z  \
 0   -0.116180  0.180296 -0.486673 -0.309556 -0.044906  0.881523 -0.245756   
 1   -0.102088  0.083932 -0.500527 -0.382532  0.033438  0.658307 -0.230434   
 2   -0.074554  0.230737 -0.462754 -0.158739  0.244184  1.090140 -0.029059   
 3   -0.063818  0.191270 -0.480363 -0.105224  0.343683  0.826240 -0.237000   
 4   -0.057705  0.023182 -0.451117 -0.020085  0.259070  0.539398 -0.002793   
 ..        ...       ...       ...       ...       ...       ...       ...   
 96  -0.164536 -0.205538  0.634188  0.398309 -0.194542 -0.224132  1.380561   
 97  -0.125568 -0.176311  0.139748 -0.399559  0.020120 -0.117738  0.973435   
 98  -0.151367 -0.182936 -0.110785 -0.044411 -0.151453 -0.071843  0.763305   
 99  -0.143221 -0.166673  0.155962  0.101541 -0.108364 -0.047851  0.800516   
 100 -0.157005 -0.185252 -0.153279  0.415337 -0.214912 -0.112523  0.553175   
 
        Volume  footprintDensity  volumeDensity     label  
 0

In [192]:
dbscan_cluster

[        varX      varY      varZ  median_Z   range_X   range_Y   range_Z  \
 0  -0.157509 -0.228395 -0.552459 -0.562538 -0.381000 -0.723763 -0.935244   
 1  -0.163223 -0.227765 -0.561238 -0.095494 -0.409989 -0.718545 -0.968076   
 2  -0.164314 -0.228112 -0.570005 -0.599026 -0.438192 -0.706029 -1.011853   
 3  -0.178603 -0.209263 -0.570423 -0.571052 -0.670091 -0.480728 -1.033742   
 4  -0.165057 -0.226479 -0.569615 -0.562538 -0.427225 -0.694557 -1.051253   
 ..       ...       ...       ...       ...       ...       ...       ...   
 84 -0.166164 -0.222710 -0.570414 -0.061438 -0.453862 -0.579818 -1.077519   
 85 -0.164608 -0.229564 -0.566185  0.302224 -0.473447 -0.752968 -1.007476   
 86 -0.172547 -0.230171 -0.576030 -0.907957 -0.534556 -0.725849 -1.145373   
 87 -0.117034 -0.206911 -0.568059 -1.128101 -0.122466 -0.493243 -0.902411   
 88 -0.167284 -0.230256 -0.565030  0.176949 -0.483632 -0.735238 -1.049064   
 
       Volume  footprintDensity  volumeDensity  label  
 0  -0.297704     

In [187]:
dbscan_cluster[0]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
0,-0.157509,-0.228395,-0.552459,-0.562538,-0.381000,-0.723763,-0.935244,-0.297704,0.223630,1.143596,car
1,-0.163223,-0.227765,-0.561238,-0.095494,-0.409989,-0.718545,-0.968076,-0.297533,0.409491,1.222249,car
2,-0.164314,-0.228112,-0.570005,-0.599026,-0.438192,-0.706029,-1.011853,-0.298794,0.381458,1.617044,car
3,-0.178603,-0.209263,-0.570423,-0.571052,-0.670091,-0.480728,-1.033742,-0.300353,0.241214,1.711475,car
4,-0.165057,-0.226479,-0.569615,-0.562538,-0.427225,-0.694557,-1.051253,-0.298866,0.342126,1.798972,car
...,...,...,...,...,...,...,...,...,...,...,...
84,-0.166164,-0.222710,-0.570414,-0.061438,-0.453862,-0.579818,-1.077519,-0.301160,-0.975762,0.857048,car
85,-0.164608,-0.229564,-0.566185,0.302224,-0.473447,-0.752968,-1.007476,-0.301088,-0.179600,1.151979,car
86,-0.172547,-0.230171,-0.576030,-0.907957,-0.534556,-0.725849,-1.145373,-0.303579,-0.742728,2.444993,car
87,-0.117034,-0.206911,-0.568059,-1.128101,-0.122466,-0.493243,-0.902411,-0.300771,-1.268524,0.918189,fence


In [188]:
dbscan_cluster[1]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
0,-0.176374,-0.223601,1.553266,1.354292,-0.507918,-0.450475,1.459360,-0.254654,-0.975654,-0.900638,pole
1,-0.174270,-0.222286,1.658843,1.471053,-0.489115,-0.489072,1.454982,-0.263938,-0.870408,-0.830884,pole
2,-0.171121,-0.221118,1.703944,1.382266,-0.441326,-0.526622,1.485626,-0.262633,-0.863773,-0.826390,pole
3,-0.171687,-0.222149,1.793674,1.215637,-0.460912,-0.505758,1.371806,-0.268515,-1.056220,-0.866065,pole
4,-0.164046,-0.202627,1.507739,1.115904,-0.323810,-0.591290,1.006267,-0.251589,-1.073353,-0.923273,pole
5,-0.177340,-0.224131,1.535047,1.322669,-0.533772,-0.538096,1.562236,-0.265531,-0.680014,-0.822353,pole
6,-0.178392,-0.225518,1.780234,1.478350,-0.530640,-0.621540,1.527214,-0.278885,-0.585439,-0.737251,pole
7,-0.178777,-0.224653,1.857818,1.256990,-0.586263,-0.547483,1.525025,-0.277520,-0.576394,-0.747169,pole
8,-0.172737,-0.220797,1.691474,1.901610,-0.464046,-0.469253,1.717644,-0.258602,-0.908447,-0.846651,pole
9,-0.178908,-0.225196,1.103379,1.264288,-0.544741,-0.462995,0.936224,-0.260755,-0.876087,-0.874048,pole


In [189]:
dbscan_cluster[2]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
0,-0.146841,-0.160230,0.061162,-0.258473,-0.048038,0.026206,1.069744,0.010641,1.256963,-0.455714,tree
1,-0.125568,-0.176311,0.139748,-0.399559,0.020120,-0.117738,0.973435,0.008087,1.274120,-0.494122,tree
2,-0.151367,-0.182936,-0.110785,-0.044411,-0.151453,-0.071843,0.763305,-0.116755,1.307658,-0.249982,tree
3,-0.161568,-0.178286,0.026934,0.206140,-0.294822,-0.116694,1.004079,-0.139188,1.321384,-0.366975,tree
4,-0.159128,-0.172558,0.119175,-0.110089,-0.211778,-0.004044,1.023778,-0.115347,0.846751,-0.398260,tree
...,...,...,...,...,...,...,...,...,...,...,...
96,-0.009115,0.085645,0.015126,-0.270635,0.733052,0.981661,0.397767,0.560738,0.083682,-0.555704,building
97,-0.103142,0.182745,-0.512068,-0.453075,0.125885,0.947240,-0.018115,0.058107,0.056504,-0.341729,building
98,-0.003959,0.122134,-0.117726,0.466420,0.430643,0.797034,0.581630,0.663012,0.708896,-0.605965,building
99,-0.009724,0.104511,-0.196479,0.848326,0.492535,0.733408,0.347423,0.634529,0.485685,-0.634648,building


In [190]:
dbscan_cluster[3]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
0,-0.051888,-0.220086,-0.570986,-1.001609,0.067127,-0.541225,-0.943999,-0.289032,-1.317983,-0.582225,fence
1,-0.020014,-0.226327,-0.561531,-1.336082,0.141553,-0.563129,-0.808290,-0.276482,-1.231539,-0.689856,fence
2,-0.025739,-0.215739,-0.567836,-1.316621,0.248887,-0.515148,-0.744814,-0.275533,-1.230619,-0.608807,fence
3,0.007454,-0.232430,-0.548747,-1.049044,0.436910,-0.743581,-0.714170,-0.278407,-0.821744,-0.441584,fence
4,0.061891,-0.217030,-0.557987,-0.898227,0.700931,-0.446305,-0.799535,-0.279721,-1.304521,-0.387059,fence
5,0.231407,-0.225873,-0.556151,-0.951743,0.796511,-0.435874,-0.954943,-0.246747,-1.220109,-0.641471,fence
6,-0.091978,-0.232055,-0.566039,-1.180400,0.101597,-0.730021,-0.913355,-0.294400,-1.038021,-0.190796,fence
7,-0.002667,-0.231001,-0.558463,-0.555241,0.123535,-0.750883,-1.003098,-0.293702,-1.149359,-0.473100,fence
8,0.028202,-0.229783,-0.562797,-0.791196,0.358566,-0.672653,-0.981209,-0.283504,-0.945273,-0.245191,fence
9,0.047769,-0.232935,-0.558100,-1.149993,0.713467,-0.738367,-0.965888,-0.288269,-1.143396,-0.368709,fence


In [191]:
dbscan_cluster[4]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
0,-0.179002,-0.205315,-0.565706,0.684131,-0.689677,-0.479684,-0.989965,-0.300054,0.495942,1.579396,car
1,-0.163085,-0.227107,-0.566555,0.682914,-0.431142,-0.703944,-0.961510,-0.298652,0.550546,1.861294,car
2,-0.178768,-0.198209,-0.568329,0.517503,-0.661473,-0.388938,-0.961510,-0.299167,0.075355,1.457934,car
3,-0.178474,-0.193409,-0.565331,0.799676,-0.655989,-0.370160,-0.928677,-0.298825,0.377337,1.943968,car
4,-0.178580,-0.200622,-0.556383,0.906707,-0.652856,-0.426487,-0.895844,-0.299353,-0.005401,1.354010,car


# Validation

In [208]:
def pair(m):
    pair_idx = []
    for i in range(0, m):
        for j in range(0,m):
            if i<j:
                pair_idx.append([i,j])

    return pair_idx

In [275]:
def external_index(ss, sd, ds, dd):
    # step4: compute external indicator
    jc = ss / (ss+sd+ds) # Jaccard Coefficient
    fmi = ss / math.sqrt((ss+sd)*(ss+ds)) # Fowlkes and Mallows
    ri = (ss+dd) / (ss+sd+ds+dd) # Rand Index

    return {'jc': jc, 'fmi': fmi, 'ri': 'ri'}

In [273]:
def validation(cluster):
    # step1: merge all clusters to a validDF
    validDF = pd.DataFrame(columns = ['actual_label','predict_label'])

    for cls in cluster:
        addDF = pd.DataFrame(cls.loc[:,'label'])
        addDF.rename(columns={'label':'actual_label'}, inplace = True)
        addDF['predict_label'] = [cls['label'].value_counts().idxmax()] * len(cls)

        validDF = pd.concat([validDF, addDF], sort=False, ignore_index=True)

    # step2: pair up
    pair_idx = pair(len(validDF))

    # step3: go through all possibilities for each pair
    ss, sd, ds, dd = 0, 0, 0, 0
    for [i,j] in pair_idx:
        i_act, i_pre = validDF['actual_label'][i], validDF['predict_label'][i]
        j_act, j_pre = validDF['actual_label'][j], validDF['predict_label'][j]
        if i_act == i_pre and j_act == j_pre:
            ss += 1
        elif i_act == i_pre and j_act != j_pre:
            sd += 1
        elif i_act != i_pre and j_act == j_pre:
            ds += 1
        elif i_act != i_pre and j_act != j_pre:
            dd += 1

    # step4: compute external indicator
    external_indicator = external_index(ss, sd, ds, dd)

    return external_indicator